### load package
***

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import igraph as ig
import cairocffi
import cairo
import networkx as nx
from tqdm.auto import tqdm
from scipy.spatial import Delaunay

## 2. Let’s Help Santa!
***
Companies like Google and Uber have a vast amount of statistics about transportation dynamics. Santa has decided to use network theory to facilitate his gift delivery for the next Christmas. When we learned about his decision, we designed this part of the project to help him. We will send him your results for this part!

### 1. Download the Data
***
Go to [“Uber Movement”](https://movement.uber.com/?lang=en-US) website and download data of Travel Times by Month (All Days), 2019 Quarter 4, for Los Angeles area1. The dataset contains pairwise traveling time statistics between most pairs of points in the Los Angeles area. Points on the map are represented by unique IDs. To understand the correspondence between map IDs and areas, download Geo Boundaries file from the same website2. This file contains latitudes and longitudes of the corners of the polygons circumscribing each area. To be specific, if an area is represented by a polygon with 5 corners, then you have a 5 × 2 matrix of the latitudes and longitudes, each row of which represents latitude and longitude of one corner. We recommend doing this part of the project (Q9 - Q18) in Python.

### 2. Build Your Graph
***
Read the dataset at hand, and build a graph in which nodes correspond to locations, and undirected weighted edges correspond to the mean traveling times between each pair of locations (only December). Add the centroid coordinates of each polygon region (a 2-D vector) as an attribute to the corresponding vertex. The graph will contain some isolated nodes (extra nodes existing in the Geo Boundaries JSON file) and a few small connected components. Remove such nodes and just keep the largest connected component of the graph. In addition, merge duplicate edges by averaging their weights 3. We will refer to this cleaned graph as G afterwards.

#### QUESTION 9: Report the number of nodes and edges in G.

> Ans: For G, the number of nodes is **2649** and the number of edges is **1004955**.

In [2]:
df = pd.read_csv("los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv")
df = df[df["month"]==12][["sourceid", "dstid", "mean_travel_time"]].reset_index(drop=True)

df["source_id"] = df[["sourceid", "dstid"]].min(axis=1)
df["dst_id"] = df[["sourceid", "dstid"]].max(axis=1)
graph = df.groupby(by=["source_id", "dst_id"], as_index=False)["mean_travel_time"].mean()

with open('graph_data.txt','w') as f:
    for _, (source_id, dst_id, mean_travel_time) in enumerate(graph.values):
        f.write('{} {} {}\n'.format(int(source_id), int(dst_id), mean_travel_time))

g = ig.Graph.Read(f='graph_data.txt', format='ncol', directed=False)
gcc = g.components().giant()

print("Number of nodes: ", len(gcc.vs))
print("Number of edges: ", len(gcc.es))

Number of nodes:  2649
Number of edges:  1004955


### 3. Traveling Salesman Problem
***
#### QUESTION 10: Build a minimum spanning tree (MST) of graph G. Report the street addresses near the two endpoints (the centroid locations) of a few edges. Are the results intuitive?

> Ans:

In [3]:
# build MST
mst = gcc.spanning_tree(weights=gcc.es["weight"])
ig.plot(mst,**{"vertex_size": 3})

# load location data
loc_data = {}
with open('los_angeles_censustracts.json', 'r') as f:
    cur = json.loads(f.readline())
    for feature in cur['features']:
        coords = np.array(feature['geometry']['coordinates'][0][0])
        loc_data[feature['properties']['MOVEMENT_ID']] = {'address': feature['properties']['DISPLAY_NAME'],
                                                          'mean_coords': np.mean(coords.reshape(-1, 2), axis=0)}

# generate the street addresses for 5 edges
mst_edges = mst.es()
print(" The street addresses near the two endpoints (the centroid locations) of 5 edges:")
for edge in mst_edges[:5]:
    x, y = mst.vs(edge.tuple[0])[0]['name'], mst.vs(edge.tuple[1])[0]['name']
    print(loc_data[str(x)]['address'], loc_data[str(x)]['mean_coords'], 
          loc_data[str(y)]['address'], loc_data[str(y)]['mean_coords'])

 The street addresses near the two endpoints (the centroid locations) of 5 edges:
Census Tract 480302 [-118.11683    34.107225] Census Tract 480304 [-118.134532   34.091566]
Census Tract 480302 [-118.11683    34.107225] Census Tract 481002 [-118.113375   34.09784 ]
Census Tract 480303 [-118.134669   34.098771] Census Tract 480304 [-118.134532   34.091566]
Census Tract 480303 [-118.134669   34.098771] Census Tract 480400 [-118.12276   34.10447]
Census Tract 480303 [-118.134669   34.098771] Census Tract 480901 [-118.144793   34.080843]


#### QUESTION 11: Determine what percentage of triangles in the graph (sets of 3 points on the map) satisfy the triangle inequality. You do not need to inspect all triangles, you can just estimate by random sampling of 1000 triangles.

> Ans: **0.914**.

In [4]:
triangles = []
while len(triangles)<1000:
    pts = np.random.randint(1, len(gcc.vs), size=3)
    try:
        e1 = gcc.get_eid(pts[0], pts[1])
        e2 = gcc.get_eid(pts[1],pts[2])
        e3 = gcc.get_eid(pts[2],pts[0])
        
        w1 = gcc.es['weight'][e1]
        w2 = gcc.es['weight'][e2]
        w3 = gcc.es['weight'][e3]
        
        if w1+w2>w3 and w1+w3>w2 and w3+w2>w1: 
            triangles.append(1)
        else:
            triangles.append(0)
    except:
        continue

print("The percentage of triangles in the graph satisfy the triangle inequality: ", sum(triangles)/len(triangles))

The percentage of triangles in the graph satisfy the triangle inequality:  0.909


Now, we want to find an approximation solution for the traveling salesman problem (TSP) on G. Apply the 1-approximate algorithm described in the class. Inspect the sequence of street addresses visited on the map and see if the results are intuitive.

#### QUESTION 12: Find an upper bound on the empirical performance of the approximate algorithm:
$$
ρ = \frac{\text{Approximate TSP Cost}}{\text{Optimal TSP Cost}}
$$

> Ans:

In [6]:
mst_cost = 0
mg = nx.MultiGraph()

for edge in mst.es:
    i, j = mst.vs(edge.tuple[0])[0]["name"], mst.vs(edge.tuple[1])[0]["name"]
    w = edge["weight"]
    mst_cost += w
    mg.add_edge(int(i), int(j), weight=w)
    mg.add_edge(int(j), int(i), weight=w)

mst_cost

269084.5449999994

In [7]:
1/0

ZeroDivisionError: division by zero

In [ ]:
mst1.es[0]["weight"]

In [ ]:
g = nx.from_pandas_edgelist(arr1, 'source', 'sink', ['weight'])
gcc = g.subgraph(max(nx.connected_components(g), key=len))
print(len(gcc.nodes))
mst = nx.minimum_spanning_tree(gcc)
mg = nx.MultiGraph()
mst_cost = 0
for i in mst.edges:
    w = mst.edges[i[0],i[1]]['weight']
    mst_cost += w
    mg.add_edge(i[0],i[1],weight=w)
    mg.add_edge(i[0],i[1],weight=w)
mst_cost

In [ ]:
len(mst.nodes)

In [ ]:
(graph.values-arr1.values).sum()

In [ ]:
arr1

In [ ]:
# arr = gd
# for i in range(0,len(arr)):
#     if(arr[i][0]>arr[i][1]):
#         t = arr[i][0]
#         arr[i][0] = arr[i][1]
#         arr[i][1] = t
# newdf = pd.DataFrame(arr)
# arr1 = newdf.groupby([0,1]).mean().reset_index()
# arr1 = arr1.rename(columns={0: "source", 1: "sink", 2: "weight"})

# g = nx.from_pandas_edgelist(arr1, 'source','sink', ['weight'])
# gcc = g.subgraph(max(nx.connected_components(g), key=len))
# mst = nx.minimum_spanning_tree(gcc)
# mg = nx.MultiGraph()
# mst_cost = 0
# for i in mst.edges:
#     w = mst.edges[i[0],i[1]]['weight']
#     mst_cost += w
#     mg.add_edge(i[0],i[1],weight=w)
#     mg.add_edge(i[0],i[1],weight=w)


vertices, count = [], 0
for i in mg.nodes:
    vertices.append(i)
    count += 1
    if count>60: 
        break

costs, cur_paths = [], []
for vertex in tqdm(vertices):
    tour = [u for u,v in nx.eulerian_circuit(mg,source=vertex)]
    cur_path, visited_nodes = [], set()
    for i in tour:
        if i not in visited_nodes: 
            cur_path.append(i) 
            visited_nodes.add(i) 
    cur_path.append(cur_path[0])
    cur_paths.append(cur_path)

    approx_cost = 0
    for i in range(len(cur_path)-1):
        s,t = cur_path[i], cur_path[i+1]
        w = 0
        if mst.has_edge(s,t): 
            w = mst.edges[s,t]['weight']
        else: 
            w = nx.dijkstra_path_length(gcc,s,t)
        approx_cost += w
    costs.append(approx_cost)

min_approx_cost = min(costs)
trajectory = cur_paths[np.argmin(costs)]

#### QUESTION 13: Plot the trajectory that Santa has to travel!

> Ans:

### 4. Analysing Traffic Flow
***
Next December, there is going to be a large group of visitors travelling between a location near Malibu to a location near Long Beach. We would like to analyse the maximum traffic that can flow between the two locations.

### 5. Estimate the Roads
***
We want to estimate the map of roads without using actual road datasets. Educate yourself about Delaunay triangulation algorithm and then apply it to the nodes coordinates.

#### QUESTION 14: Plot the road mesh that you obtain and explain the result. Create a graph $G_∆$ whose nodes are different locations and its edges are produced by triangulation.

> Ans:

### 6. Calculate Road Traffic Flows
***
#### QUESTION 15: Using simple math, calculate the traffic flow for each road in terms of cars/hour. Report your derivation.
Hint: Consider the following assumptions:
+ Each degree of latitude and longitude ≈ 69 miles
+ Car length ≈ 5 m = 0.003 mile
+ Cars maintain a safety distance of 2 seconds to the next car
+ Each road has 2 lanes in each direction

Assuming no traffic jam, consider the calculated traffic flow as the max capacity of each road.

> Ans:

### 7. Calculate Max Flow
***
Consider the following locations in terms of latitude and longitude:
+ Source coordinates (in Malibu): $[34.04, -118.56]$
+ Destination coordinates (in Long Beach): $[33.77, -118.18]$

#### QUESTION 16: Calculate the maximum number of cars that can commute per hour from Malibu to Long Beach. Also calculate the number of edge-disjoint paths between the two spots. Does the number of edge-disjoint paths match what you see on your road map?

> Ans:

### 8. Prune Your Graph
***
In $G_∆$, there are a number of unreal roads that could be removed. For instance, you might notice some unreal links along the concavities of the beach, as well as in the hills of Topanga. Apply a threshold on the travel time of the roads in $G_∆$ to remove the fake edges. Call the resulting graph $\tilde{G}_∆$.

#### QUESTION 17: Plot $\tilde{G}_∆$ on actual coordinates. Do you think the thresholding method worked?

> Ans:

#### QUESTION 18: Now, repeat question 16 for $\tilde{G}_∆$ and report the results. Do you see any changes? Why?

> Ans:

### 9. Construct New Roads
***
Let us assume that the output of question 17 (graph $\tilde{G}_∆$) is the actual road network of LA. The government of LA wants to construct 20 new roads (each road is an edge). They ask you to help them decide where to construct the roads. In other words, you have to create 20 new edges in the graph.

#### QUESTION 19: Strategy 1 (geo distance, static): Reduce the maximum extra travelling distance. The extra travelling distance between 2 locations is the difference of the shortest traveling distance and the straight line distance. i.e extra_distance(v,s) = distance_of_shortest_path(v,s) - euclidean_distance(v,s). Use the coordinates of v and s to get the euclidean distance between them. Calculate the extra distance between all pairs of points. The top 20 pairs with highest extra distance will be the new edges we suggest. Print the source and destination of these pairs and write them in your report. Create these new edges in the graph, and plot the resultant graph on actual coordinates. What is the time complexity of the strategy?

> Ans:

#### QUESTION 20: Strategy 2 (geo distance, static, with frequency): In strategy1, we are using the geographical distance between points to decide which new road to create. However, in the real world, some pairs are more in demand than other pairs. Assume that you know the frequency of travel between every pair. Use the frequency as the weight to multiply the difference. i.e weighted_extra_distance(v,s) = extra_distance(v,s) * frequency(v,s) where frequency(v,s) is a random integer between $[1,1000]$. Use the coordinates of v and s to get the euclidean distance between them. Calculate the weighted extra distance between all pairs of points. The top 20 pairs with highest weighted extra distance will be the new edges we suggest. Print the source and destination of these pairs and write them in your report. Create these new edges in the graph, and plot the resultant graph on actual coordinates. What is the time complexity of the strategy?

> Ans:

#### QUESTION 21: Strategy 3 (geo distance, dynamic): In the above strategy, we are creating all roads at the same time. This time, we will create the roads one by one. Repeat 20 times: i) Compute extra distance between all the pairs in the graph. ii) Create a road between the pair with highest extra distance and update the graph. iii) Print the source and destination of this new edge. Report the source and destination of the 20 new edges. plot the final graph on actual coordinates. What is the time complexity of the strategy?

> Ans:

#### QUESTION 22: Strategy 4 (Travel time, static): We want to optimize to reduce the maximum extra travelling time. The extra travelling distance between 2 locations is the difference of the shortest traveling time and the straight line travel time. i.e extra_time(v,s) = travel_time_of_shortest_path(v,s) - euclidean_distance(v,s) / travel_speed(v,s) and travel_speed(v,s) = distance_of_shortest_path(v,s) / travel_time_of_shortest_path(v,s). Use the coordinates of v and s to get the euclidean distance between them. Calculate the extra time between all pairs of points. The top 20 pairs with highest extra time will be the new edges we suggest. Print the source and destination of these pairs and write them in your report. Create these new edges in the graph, and plot the resultant graph on actual coordinates. What is the time complexity of the strategy? What is the time complexity of the strategy?

> Ans:

#### QUESTION 23: Strategy 5 (Travel time, dynamic): Similar to strategy 3, this time we will create roads one by one while optimizing the travel time. Repeat 20 times: i) Compute extra time between all the pairs in the graph. ii) Create a road between the pair with highest extra time and update the graph. iii) Print the source and destination of this new edge. Report the source and destination of the 20 new edges. plot the final graph on actual coordinates. What is the time complexity of the strategy?

> Ans:

#### QUESTION 24: Strategy comparison. Compare the following strategies:
+ a) 1 vs 2 - compare and analyze the results. which is better? why?
+ b) 1 vs 3 - compare and analyze the results. which is better? why?
+ c) 1 vs 4 - compare and analyze the results. which is better? why?
+ d) statics vs dynamic - Considering we want to improve the overall road network by constructing new roads, is either of them the optimal strategy?
    + i) if yes, which one and why?
    + ii) if not, what would be a better strategy to construct roads and is it optimal? What is the time complexity?

(assume that you want to reduce travelling distances and you know before hand that we have to
construct 20 new roads).

> Ans:

#### e) Open ended question : Come up with new strategy. You are free to make any assumptions. You don’t have to necessarily optimize traveling time or distance, you can come up with any constraints. justify your strategy.

> Ans:

### 10. Define Your Own Task
***
Brainstorm with your group, then define and implement a new interesting task based on the dataset at hand. We recommend you to download and use data of other time periods. The duration of this data should be at least three months. For example you may use January 2020 to March 2020 in which the major changes happened to daily’s commutes. This part is open-ended and you’ll be graded based on your creativity. You may also wish to go beyond, and use any other navigation- / traffic- related dataset and define your favorite task. This part is worth 20% credit of the project. You may discuss your ideas with the TA in designated office hours.

> Ans: